# Creation of composite marker from a stone


## Materials
### Localizator 

### Scene and equipements

## Method

### Module importations

In [ ]:
import os
import json
import time

import numpy as np
from cv2 import aruco
import gbu  # <- python -m pip install git+https://github.com/elmokulc/GBU_pose_classifier.git
import matplotlib.pyplot as plt

### Import image dataset

You can download the images source from [here](https://github.com/elmokulc/GBU_pose_classifier/archive/refs/tags/GBU-0.0.tar.gz)

In [ ]:
if not os.path.isfile("./GBU-0.0.tar.gz"):
    ! wget https://github.com/elmokulc/GBU_pose_classifier/archive/refs/tags/GBU-0.0.tar.gz 
if not os.path.isdir("./_dataset"):
    !tar -zxf GBU-0.0.tar.gz GBU_pose_classifier-GBU-0.0/dataset_subsample && \
    mkdir _dataset && \
    cp GBU_pose_classifier-GBU-0.0/dataset_subsample/* ./_dataset/ && \
    rm -r GBU_pose_classifier-GBU-0.0 && \
    echo "Image importation done !"

### Setting up dataset

In [ ]:
# SETUP
image_directory = "./_dataset/"

metadata = json.load(open(image_directory + "metadata.json"))
camera_matrix = np.array(metadata["camera_matrix"])
distortion_coefficients = np.array(metadata["distortion_coefficients"])
md = float(metadata["aruco_marker_size"].split()[0])
marker_dimension = {i: md for i in range(1, 250)}
marker_size = 12.4079e-3
marker_dimension = {i: marker_size for i in range(1, 250)}

# Aruco settings
parameters = aruco.DetectorParameters_create()
parameters.cornerRefinementMethod = 3
parameters.cornerRefinementWinSize = 5
parameters.cornerRefinementMaxIterations = 100

#### Individual marker pose estimation

In [ ]:
# STEP 1: PREPROCESSING
print("# BATCH INTITIALIZATION")
batch = gbu.calibration.ImageBatchCalibration(
    aruco_dict=aruco.DICT_6X6_250,
    parameters=parameters,
    marker_dimension=marker_dimension,
    output_directory="./_outputs_calibration/",
    camera_matrix=camera_matrix,
    distortion_coefficients=distortion_coefficients,
)

batch.load_image_batch(directory=image_directory)
batch.detect_markers(
    plot_markers=False, enforce=False, meta_filename="metadata.p", protocol=4
)
batch.estimate_pose()

#### Graph theory & pre-optmization educated guess

In [ ]:
data, graph, central_mk, cycles = batch.get_graph_data(
    atomic_cycle=False, enabled_central_mk=True
)
MG = gbu.utils.get_multigraph(data=data)
gbu.utils.plot_graph(G=graph, MG=MG)

In [ ]:
batch.get_good_class()
batch.graph_calibration()

In [ ]:
batch.plot_gra

#### Least-square optimization

In [ ]:
t0 = time.time()
compo, poseBatch, sol = batch.optimize_calibration()
t1 = time.time()
print(
    "=> Ran optimization in {0}s for {1} poses".format(
        (t1 - t0), len(batch.data_good_core)
    )
)

#### Reprojections error 

In [ ]:
batch.plot_reprojection_errors(
    global_plot=True, individual_plots=False, plot_type="interactive"
)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


data_projected_points = batch.projected_points()